<a href="https://colab.research.google.com/github/kaisu313/Capstone-Week-1-submission/blob/master/Vancouver_Restaurant_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PROBLEM/BUSINESS CASE:  Vancouver downtown NE and SW side are both filled with people so businesses can easily thrive there.  If I want to open up a dessert place serving unique authentic Taiwanese delicacy, which area in downtown should I setup?  **

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


<a id='item1'></a>

Downloading Vancouver location data from genames 

In [0]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

req = requests.get("https://www.geonames.org/postal-codes/CA/BC/british-columbia.html")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[2]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])

neighborhood.head(8)

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Port Moody,V3H,Canada,British Columbia,NaN,NaN
1,NaN,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863
2,2.0,Pitt Meadows,V3Y,Canada,British Columbia,NaN,NaN
3,NaN,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69
4,3.0,White Rock,V4B,Canada,British Columbia,NaN,NaN
5,NaN,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806
6,4.0,Penticton,V2A,Canada,British Columbia,NaN,NaN
7,NaN,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586


In [0]:
neighborhood=neighborhood.rename(columns={"Unnamed: 0":"Citynum"})
neighborhood.head(8)



,Citynum,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Port Moody,V3H,Canada,British Columbia,NaN,NaN
1,NaN,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863
2,2.0,Pitt Meadows,V3Y,Canada,British Columbia,NaN,NaN
3,NaN,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69
4,3.0,White Rock,V4B,Canada,British Columbia,NaN,NaN
5,NaN,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806
6,4.0,Penticton,V2A,Canada,British Columbia,NaN,NaN
7,NaN,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586


In [0]:
van=neighborhood[neighborhood.Country == "Canada"]
van.head()


,Citynum,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Port Moody,V3H,Canada,British Columbia,NaN,NaN
2,2.0,Pitt Meadows,V3Y,Canada,British Columbia,NaN,NaN
4,3.0,White Rock,V4B,Canada,British Columbia,NaN,NaN
6,4.0,Penticton,V2A,Canada,British Columbia,NaN,NaN
8,5.0,Westbank,V4T,Canada,British Columbia,NaN,NaN


In [0]:
cord=neighborhood[~(neighborhood.Country == "Canada")]
cord=cord.drop(384)
cord.head()


,Citynum,Place,Code,Country,Admin1,Admin2,Admin3
1,NaN,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863,49.323/-122.863
3,NaN,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69,49.221/-122.69
5,NaN,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806,49.026/-122.806
7,NaN,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586,49.481/-119.586
9,NaN,49.866/-119.739,49.866/-119.739,49.866/-119.739,49.866/-119.739,49.866/-119.739,49.866/-119.739


In [0]:
van['Lat']=cord['Place'].values
van.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Citynum,Place,Code,Country,Admin1,Admin2,Admin3,Lat
0,1.0,Port Moody,V3H,Canada,British Columbia,NaN,NaN,49.323/-122.863
2,2.0,Pitt Meadows,V3Y,Canada,British Columbia,NaN,NaN,49.221/-122.69
4,3.0,White Rock,V4B,Canada,British Columbia,NaN,NaN,49.026/-122.806
6,4.0,Penticton,V2A,Canada,British Columbia,NaN,NaN,49.481/-119.586
8,5.0,Westbank,V4T,Canada,British Columbia,NaN,NaN,49.866/-119.739


In [0]:
van.drop(columns=["Citynum","Admin2","Admin3"], axis=1,inplace=True)

van.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Place,Code,Country,Admin1,Lat
0,Port Moody,V3H,Canada,British Columbia,49.323/-122.863
2,Pitt Meadows,V3Y,Canada,British Columbia,49.221/-122.69
4,White Rock,V4B,Canada,British Columbia,49.026/-122.806
6,Penticton,V2A,Canada,British Columbia,49.481/-119.586
8,Westbank,V4T,Canada,British Columbia,49.866/-119.739


In [0]:
coordinates=van["Lat"].str.split("/",expand=True)
coordinates.head()

,0,1
0,49.323,-122.863
2,49.221,-122.69
4,49.026,-122.806
6,49.481,-119.586
8,49.866,-119.739


In [0]:
van['Latitude']=coordinates[0]
van['Longitude']=coordinates[1]
van.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Place,Code,Country,Admin1,Lat,Latitude,Longitude
0,Port Moody,V3H,Canada,British Columbia,49.323/-122.863,49.323,-122.863
2,Pitt Meadows,V3Y,Canada,British Columbia,49.221/-122.69,49.221,-122.69
4,White Rock,V4B,Canada,British Columbia,49.026/-122.806,49.026,-122.806
6,Penticton,V2A,Canada,British Columbia,49.481/-119.586,49.481,-119.586
8,Westbank,V4T,Canada,British Columbia,49.866/-119.739,49.866,-119.739


In [0]:
van.drop(columns=["Lat"], axis=1,inplace=True)

van.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Place,Code,Country,Admin1,Latitude,Longitude
0,Port Moody,V3H,Canada,British Columbia,49.323,-122.863
2,Pitt Meadows,V3Y,Canada,British Columbia,49.221,-122.69
4,White Rock,V4B,Canada,British Columbia,49.026,-122.806
6,Penticton,V2A,Canada,British Columbia,49.481,-119.586
8,Westbank,V4T,Canada,British Columbia,49.866,-119.739


In [0]:
van=van.rename(columns={"Admin1":"Province"})
van

,Place,Code,Country,Province,Latitude,Longitude
0,Port Moody,V3H,Canada,British Columbia,49.323,-122.863
2,Pitt Meadows,V3Y,Canada,British Columbia,49.221,-122.69
4,White Rock,V4B,Canada,British Columbia,49.026,-122.806
6,Penticton,V2A,Canada,British Columbia,49.481,-119.586
8,Westbank,V4T,Canada,British Columbia,49.866,-119.739
10,Winfield,V4V,Canada,British Columbia,50.022,-119.405
12,Kimberley,V1A,Canada,British Columbia,49.683,-115.986
14,Saltspring Island,V8K,Canada,British Columbia,48.814,-123.497
16,Northern British Columbia (Fort Nelson),V0C,Canada,British Columbia,58.387,-125.717
18,Central Okanagan and High Country (Revelstoke),V0E,Canada,British Columbia,51.505,-119.203


#### Load and explore the data

#### Use geopy library to get the latitude and longitude values 

In [0]:
address = 'Vancouver, BC'

geolocator = Nominatim(user_agent="van_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.2608724, -123.1139529.


#### Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = 'ZPWRQK2VHXI0TDUY4ZPHIOAFE4FXMW4RRYXPKUURC5HO1XLL' # your Foursquare ID
CLIENT_SECRET = 'MPTMT1EURSPUOQCSWJQ5FBCONZZMI5EDSFSCEJWE3H2IXWVX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZPWRQK2VHXI0TDUY4ZPHIOAFE4FXMW4RRYXPKUURC5HO1XLL
CLIENT_SECRET:MPTMT1EURSPUOQCSWJQ5FBCONZZMI5EDSFSCEJWE3H2IXWVX


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [0]:
van.loc[230, 'Place']

'Vancouver (SW Downtown)'

Get the neighborhood's latitude and longitude values.

In [0]:
neighborhood_latitude = van.loc[230, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = van.loc[230, 'Longitude'] # neighborhood longitude value

neighborhood_name = van.loc[230, 'Place'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Vancouver (SW Downtown) are 49.277, -123.13.


#### Now, let's get the top 100 venues within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [0]:

LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL






'https://api.foursquare.com/v2/venues/explore?&client_id=ZPWRQK2VHXI0TDUY4ZPHIOAFE4FXMW4RRYXPKUURC5HO1XLL&client_secret=MPTMT1EURSPUOQCSWJQ5FBCONZZMI5EDSFSCEJWE3H2IXWVX&v=20180605&ll=49.277,-123.13&radius=500&limit=100'

Double-click __here__ for the solution.
<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
    
url # display URL
--> 

Send the GET request and examine the resutls

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8394da78a484001b181ac1'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50183062e4b090136c898ce0-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1fa931735',
         'name': 'Hotel',
         'pluralName': 'Hotels',
         'primary': True,
         'shortName': 'Hotel'}],
       'id': '50183062e4b090136c898ce0',
       'location': {'address': '1234 Hornby Street',
        'cc': 'CA',
        'city': 'Vancouver',
        'country': 'Canada',
        'distance': 141,
        'formattedAddress': ['1234 Hornby Street',
         'Vancouver BC V6Z 1W2',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 49.2779047,
          '

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [0]:
venues = results['response']['groups'][0]['items']
venues

  

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-0-50183062e4b090136c898ce0-0',
  'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1fa931735',
     'name': 'Hotel',
     'pluralName': 'Hotels',
     'primary': True,
     'shortName': 'Hotel'}],
   'id': '50183062e4b090136c898ce0',
   'location': {'address': '1234 Hornby Street',
    'cc': 'CA',
    'city': 'Vancouver',
    'country': 'Canada',
    'distance': 141,
    'formattedAddress': ['1234 Hornby Street',
     'Vancouver BC V6Z 1W2',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 49.2779047,
      'lng': -123.1286238}],
    'lat': 49.2779047,
    'lng': -123.1286238,
    'postalCode': 'V6Z 1W2',
    'state': 'BC'},
   'name': 'Residence Inn by Marriott Vancouver Downtown',
   'photos': {

In [0]:
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Residence Inn by Marriott Vancouver Downtown,Hotel,49.277905,-123.128624
1,Musette Caffè,Café,49.277813,-123.131349
2,Body Energy Club,Gym / Fitness Center,49.277682,-123.126894
3,Number e food,Sandwich Place,49.277899,-123.131060
4,Breka Bakery & Cafe,Bakery,49.278496,-123.128062


And how many venues were returned by Foursquare?

In [0]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

65 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in Vancouver Downtown South Side

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [0]:
# type your answer here

van_venues = getNearbyVenues(names=van['Place'],
                                   latitudes=van['Latitude'],
                                   longitudes=van['Longitude']
                                  )



Port Moody
Pitt Meadows
White Rock
Penticton
Westbank
Winfield
Kimberley
Saltspring Island
Northern British Columbia (Fort Nelson)
Central Okanagan and High Country (Revelstoke)
West Kootenays (Rossland)
South Okanagan (Summerland)
Omineca and Yellowhead (Smithers)
Cariboo and West Okanagan (100 Mile House)
Chilcotin (Alexis Creek)
Harrison Lake Region (Agassiz)
North Island, Sunshine Coast, and Southern Gulf Islands (Whistler)
North Central Island and Bute Inlet Region (Gold River)
Central Island (Chemainus)
Juan de Fuca Shore (Sooke)
Inside Passage and the Queen Charlottes (Queen Charlotte City)
Lower Skeena (Port Edward)
Atlin Region (Atlin)
Similkameen (Hope)
Vernon West
Fort St. John
Nelson
Langley Township North
Kelowna East
Kamloops Southwest
Vernon Central
Kelowna North
Kelowna Southwest
Kelowna East Central
Kelowna Central
Kelowna West
Kamloops Central and Southeast
Kamloops South and West
Kamloops North
Quesnel
Prince George North
Prince George East Central
Prince George West

Double-click __here__ for the solution.
<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 

#### Let's check the size of the resulting dataframe

In [0]:
print(van_venues.shape)
van_venues.head()

(1829, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Port Moody,49.323,-122.863,Buntzen Lake,49.324467,-122.856549,Lake
1,Pitt Meadows,49.221,-122.69,Jolly Coachman,49.222584,-122.690512,Pub
2,Pitt Meadows,49.221,-122.69,Waves Coffee House,49.222068,-122.689869,Coffee Shop
3,Pitt Meadows,49.221,-122.69,Foamers' Folly Brewing Co.,49.224774,-122.689383,Brewery
4,Pitt Meadows,49.221,-122.69,GoodLife Fitness,49.220698,-122.688491,Gym / Fitness Center


Let's check how many venues were returned for each neighborhood

In [0]:
van_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford Southeast,5,5,5,5,5,5
Abbotsford Southwest,2,2,2,2,2,2
Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley),5,5,5,5,5,5
Burnaby (Cascade-Schou / Douglas-Gilpin),4,4,4,4,4,4
Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong),7,7,7,7,7,7
Burnaby (Government Road / Lake City / SFU / Burnaby Mountain),3,3,3,3,3,3
Burnaby (Lakeview-Mayfield / Richmond Park / Kingsway-Beresford),2,2,2,2,2,2
Burnaby (Maywood / Marlborough / Oakalla / Windsor),36,36,36,36,36,36
Burnaby (Parkcrest-Aubrey / Ardingley-Sprott),10,10,10,10,10,10


#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(van_venues['Venue Category'].unique())))

There are 264 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [0]:
# one hot encoding
van_onehot = pd.get_dummies(van_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
van_onehot['Neighborhood'] = van_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [van_onehot.columns[-1]] + list(van_onehot.columns[:-1])
van_onehot = van_onehot[fixed_columns]

van_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Physical Therapist,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Ski Chalet,Ski Lodge,Ski Trail,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trade School,Trail,Travel Lounge,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Port Moody,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [0]:
van_onehot.shape

(1829, 265)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
van_grouped = van_onehot.groupby('Neighborhood').mean().reset_index()
van_grouped

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,Apres Ski Bar,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Physical Therapist,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Ski Chalet,Ski Lodge,Ski Trail,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trade School,Trail,Travel Lounge,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Abbotsford Southeast,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000

#### Let's confirm the new size

In [0]:
van_grouped.shape

(130, 265)

#### Let's print each neighborhood along with the top 5 most common venues

In [0]:
num_top_venues = 5

for hood in van_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = van_grouped[van_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbotsford Southeast----
                  venue  freq
0            Playground   0.2
1  Gym / Fitness Center   0.2
2            Restaurant   0.2
3               Brewery   0.2
4           Coffee Shop   0.2


----Abbotsford Southwest----
                        venue  freq
0  Construction & Landscaping   0.5
1                     Factory   0.5
2                     Airport   0.0
3             Motorcycle Shop   0.0
4                 Music Store   0.0


----Burnaby (Burnaby Heights / Willingdon Heights / West Central Valley)----
           venue  freq
0   Home Service   0.2
1           Café   0.2
2   Tennis Court   0.2
3  Garden Center   0.2
4           Park   0.2


----Burnaby (Cascade-Schou / Douglas-Gilpin)----
         venue  freq
0    Speakeasy  0.25
1  Auto Garage  0.25
2     Pharmacy  0.25
3         Park  0.25
4      Airport  0.00


----Burnaby (East Big Bend / Stride Avenue / Edmonds / Cariboo-Armstrong)----
               venue  freq
0  Indian Restaurant  0.29
1        Art Gal

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = van_grouped['Neighborhood']

for ind in np.arange(van_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(van_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford Southeast,Playground,Gym / Fitness Center,Brewery,Restaurant,Coffee Shop,Convenience Store,Dumpling Restaurant,Fast Food Restaurant,Comic Shop,Farmers Market
1,Abbotsford Southwest,Construction & Landscaping,Factory,Yoga Studio,Fair,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,Burnaby (Burnaby Heights / Willingdon Heights ...,Café,Home Service,Tennis Court,Park,Garden Center,Ethiopian Restaurant,Dumpling Restaurant,Duty-free Shop,Electronics Store,Elementary School
3,Burnaby (Cascade-Schou / Douglas-Gilpin),Pharmacy,Speakeasy,Auto Garage,Park,Yoga Studio,Event Space,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant
4,Burnaby (East Big Bend / Stride Avenue / Edmon...,Indian Restaurant,Art Gallery,Pool,Motel,Dog Run,Pet Store,Farmers Market,Farm,Falafel Restaurant,Fair
5,Burnaby (Government Road / Lake City / SFU / B...,Golf Course,Golf Driving Range,Burger Joint,Yoga Studio,Event Space,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Factory
6,Burnaby (Lakeview-Mayfield / Richmond Park / K...,Bus Stop,Park,Yoga Studio,Factory,Duty-free Shop,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Fair
7,Burnaby (Maywood / Marlborough / Oakalla / Win...,Coffee Shop,Pharmacy,Thai Restaurant,Sushi Restaurant,Toy / Game Store,Bookstore,Chinese Restaurant,Fast Food Restaurant,American Restaurant,Hotpot Restaurant
8,Burnaby (Parkcrest-Aubrey / Ardingley-Sprott),Coffee Shop,Convenience Store,Stadium,Bus Stop,Sushi Restaurant,Italian Restaurant,Golf Course,Martial Arts Dojo,Vietnamese Restaurant,Park
9,Burnaby (Suncrest / Sussex-Nelson / Clinton-Gl...,Garden Center,Farmers Market,Golf Course,Restaurant,Elementary School,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Yoga Studio


In [0]:
vandowntownSW=neighborhoods_venues_sorted.iloc[107]
vandowntownSW

Neighborhood              Vancouver (SW Downtown)
1st Most Common Venue                       Hotel
2nd Most Common Venue          Mexican Restaurant
3rd Most Common Venue                      Bakery
4th Most Common Venue                 Coffee Shop
5th Most Common Venue                        Café
6th Most Common Venue            Sushi Restaurant
7th Most Common Venue                   Nightclub
8th Most Common Venue                Dessert Shop
9th Most Common Venue                     Gay Bar
10th Most Common Venue           Greek Restaurant
Name: 107, dtype: object

In [0]:
vandowntownNE=neighborhoods_venues_sorted.iloc[96]
vandowntownNE

Neighborhood              Vancouver (NE Downtown / Harbour Centre / Gast...
1st Most Common Venue                                                 Hotel
2nd Most Common Venue                                            Restaurant
3rd Most Common Venue                                           Coffee Shop
4th Most Common Venue                                           Pizza Place
5th Most Common Venue                                          Hockey Arena
6th Most Common Venue                                        Clothing Store
7th Most Common Venue                                            Taco Place
8th Most Common Venue                                               Theater
9th Most Common Venue                                      Ramen Restaurant
10th Most Common Venue                                       Breakfast Spot
Name: 96, dtype: object

<a id='item4'></a>

**Unlike Vancouver Downtown SW, Vancouver Downtown NE does not have a dessert shop in its top 10 most common avenue.  Hence, a dessert shop there will be a potentially better place to start a business!**
